# RQ3 not imputed data

In [ ]:
# Imports
%matplotlib inline
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math
import csv
from ast import literal_eval
# Jupyter configurations
pd.options.display.max_columns = 2000

# Imports 
import itertools 


# Stats
from scipy.stats import mannwhitneyu
from statsmodels.sandbox.stats.multicomp import multipletests
import cliffsDelta # implementation

# Graphics 
import seaborn as sns
import matplotlib.pyplot as plt

# Auxiliar functions
from aux import *

In [ ]:
# Read all the data after fix: num_installs, macro android version,has, what's new, clean dev address
all_apps = pd.read_csv("PATH.csv", delimiter="\t", encoding='utf-8', engine= 'python',converters={'_id':str})

In [ ]:
all_apps.shape

In [ ]:
all_apps.columns

### Get unique values - basic information

In [ ]:
# Unique values 
countries = all_apps.country.unique()
top = all_apps.top.unique()
clean_category = all_apps.clean_category.unique()


# Retrieve different dates as str
retrieved_date_data = all_apps.retrieved_date_end.unique()
retrieved_date_data.sort()

# List combination between tops and category
clean_cat_no_editorChoice = np.delete(clean_category, np.where(clean_category == "editorChoice"))

list_combinations = [top[0:2], clean_cat_no_editorChoice]
top_category_combination = [p for p in itertools.product(*list_combinations)]

top_category_combination.append(('editorChoice','')) # list with all the possible top and category combinations

### set color palette

In [ ]:
# Set colors for grpahics
seq_col_brew = sns.color_palette("Greys_r", 5)
sns.set_palette(seq_col_brew)

## Create dictionaries for each of the 268 possible combinations

In [ ]:
#create a data frame dictionary to store data frames by countries
df_countries = {elem : pd.DataFrame for elem in countries}
for country in df_countries.keys():
    df_countries[country] = all_apps[:][all_apps.country == country]

#Dict for each country and category
dict_co_cate = {elem : pd.DataFrame for elem in top_category_combination}
dict_usa_cate = {elem : pd.DataFrame for elem in top_category_combination}
dict_br_cate = {elem : pd.DataFrame for elem in top_category_combination}
dict_de_cate = {elem : pd.DataFrame for elem in top_category_combination}

#create 67 dataframes for each country
create_dataFrames_by_category(df_countries['co'], dict_co_cate)
create_dataFrames_by_category(df_countries['us'], dict_usa_cate)
create_dataFrames_by_category(df_countries['br'], dict_br_cate)
create_dataFrames_by_category(df_countries['de'], dict_de_cate)

# Pivot Tables

## TLO

In [ ]:
#missing weeks summary
missing_apps_cat_country = pd.DataFrame(index= top_category_combination, dtype= str, columns= countries)
missing_num_cat_country = pd.DataFrame(index= top_category_combination, columns= countries, data= np.zeros((len(top_category_combination), len(countries))))

In [ ]:
# df, number -> df gives a dictionary with the pivot categories vs apps (1 appears, 0 not appears, NAN missing value)
dict_pivot_cat_br, incomplete_br =  pivot_sum_top(dict_br_cate,retrieved_date_data,'br',missing_num_cat_country,missing_apps_cat_country)
dict_pivot_cat_co, incomplete_co =  pivot_sum_top(dict_co_cate,retrieved_date_data,'co',missing_num_cat_country,missing_apps_cat_country)
dict_pivot_cat_de, incomplete_de =  pivot_sum_top(dict_de_cate,retrieved_date_data,'de',missing_num_cat_country,missing_apps_cat_country)
dict_pivot_cat_usa, incomplete_usa =  pivot_sum_top(dict_usa_cate,retrieved_date_data,'us',missing_num_cat_country,missing_apps_cat_country)

In [ ]:
print(incomplete_br, incomplete_co, incomplete_de, incomplete_usa)

In [ ]:
def create_df_occurrences(dict_country, country, final_list):

    for (top,category), data in  dict_country.items():
        df = pd.DataFrame(columns=['Top','Category','Occurrences','Country'])

        df['Occurrences']= data['occurrences']
        df['Country']= country.upper()
        df['Top']= top
        df['Category']= category
                                   
        final_list.append(df)
        
lst_occurences = []
create_df_occurrences(dict_pivot_cat_br,'br',lst_occurences)
create_df_occurrences(dict_pivot_cat_co,'co',lst_occurences)
create_df_occurrences(dict_pivot_cat_de,'de',lst_occurences)
create_df_occurrences(dict_pivot_cat_usa,'us',lst_occurences)

#dataframe with the information about number od occurences (how many times an app appears in the top)
df_occurences_countries = pd.concat(lst_occurences) 
df_occurences_countries['Type'] = 'TLO'

## EST

In [ ]:
#----- EST
def count_durations_by_appearance(row, count_list):
    count = 0
    appear = False

    for i in range(0,len(row)):
        
        if (not appear and not np.isnan(row[i]) and row[i]==1) :
            appear = True

        if ((np.isnan(row[i]) or row[i] ==0 ) and appear) :
            appear = False
            count_list.append(count)
            count = 0
            
        if (appear and row[i] ==1) :
            count += 1 
        
        if  (row[i]==1 and i== len(row)-1):
            count_list.append(count)


In [ ]:
# EST
def durations_rows(dict_info, dict_result,country):
    lst_dfs= []
    
    for key, dicc in dict_info.items():
        lista = []
        df_counts = pd.DataFrame(columns=['Top','Category','Count','Country'])
        data= dicc.loc[:,retrieved_date_data].values
        np.apply_along_axis(func1d=count_durations_by_appearance, axis=1, arr= data, count_list = lista)
        
        df_counts['Count']= lista 
        df_counts['Top']= key[0]
        df_counts['Category']= key[1] 
        df_counts['Country']= country.upper() 
        
        lst_dfs.append(df_counts)
    
    dict_result[country]= pd.concat(lst_dfs) 
#     return pd.concat(lst_dfs)

In [ ]:
dict_counts = {}
durations_rows(dict_pivot_cat_br, dict_counts,'br' )
durations_rows(dict_pivot_cat_co, dict_counts,'co' )
durations_rows(dict_pivot_cat_de, dict_counts,'de' )
durations_rows(dict_pivot_cat_usa, dict_counts,'us' )
df_counts_countries = pd.concat([dict_counts['br'],dict_counts['co'],dict_counts['de'],dict_counts['us']])
df_counts_countries['Type']='EST'

In [ ]:
all_apps.id.nunique()

## TBSE

In [ ]:
def count_durations_between_appearance(row, count_list):
    count = 0
    appear = False
    once = False

    for i in range(0,len(row)):
        
        if (not appear  and row[i]==1) :
            appear = True
            once=True
            
            if count !=0:
                count_list.append(count)
                count = 0  
                
        elif (np.isnan(row[i]) or row[i] ==0) and once:
            
            appear = False
            count += 1

def durations_between_appearance(dict_info, dict_result, country):
    lst_dfs= []
    
    for (top,category), dicc in dict_info.items():
        lista = []
        df_counts = pd.DataFrame(columns=['Top','Category','Between_apps','Country'])
        data= dicc.loc[:,retrieved_date_data].values
        np.apply_along_axis(func1d=count_durations_between_appearance, axis=1, arr= data, count_list = lista)

        df_counts['Between_apps']= lista 
        df_counts['Top']= top
        df_counts['Category']= category
        df_counts['Country']= country.upper() 
        lst_dfs.append(df_counts)
    
    dict_result[country]= pd.concat(lst_dfs) 

In [ ]:
dict_time_between_appearance = {}
# 
durations_between_appearance(dict_pivot_cat_br, dict_time_between_appearance,'br' )
durations_between_appearance(dict_pivot_cat_co, dict_time_between_appearance,'co' )
durations_between_appearance(dict_pivot_cat_de, dict_time_between_appearance,'de' )
durations_between_appearance(dict_pivot_cat_usa, dict_time_between_appearance,'us' )


df_time_between_countries = pd.concat([dict_time_between_appearance['br'],dict_time_between_appearance['co'],dict_time_between_appearance['de'],dict_time_between_appearance['us']])
df_time_between_countries = df_time_between_countries.reset_index(drop=True)
df_time_between_countries['Type']='TBSE'

## Stats and graphics

In [ ]:
#-------------------------------- Stats
def manWhitney_pairs(data,column,numeric_column, list_categories, c_type=None):
    p_values= {}
    u_values={}
    
    for i in range(len(list_categories)):
        par= list_categories[i]
        for j in range(i+1, len(list_categories)):
            par2 = list_categories[j]
            
            if  c_type:
                par = par.upper()
                par2= par2.upper()
                
            stat, p = mannwhitneyu(data[data[column]==par][numeric_column], data[data[column]==par2][numeric_column], alternative='two-sided')
            p_values[f'{par}___{par2}']= p
            u_values[f'{par}___{par2}']= stat
    
    return (p_values, u_values)

def cliffsDelta_pairs(data, column,numeric_column, list_categories, key_bool_vals=None, c_type=None):
    differences= {}
        
    for i in range(len(list_categories)):
        par= list_categories[i]
        for j in range(i+1, len(list_categories)):
            par2 = list_categories[j]
            if  c_type:
                par = par.upper()
                par2= par2.upper()
                
            if key_bool_vals  :
                if key_bool_vals[f'{par}___{par2}']:
                    d, res = cliffsDelta.cliffsDelta(data[data[column]==par][numeric_column], data[data[column]==par2][numeric_column])
                    differences[f'{par}___{par2}']= (res, d)
            else:
                d, res = cliffsDelta.cliffsDelta(data[data[column]==par][numeric_column], data[data[column]==par2][numeric_column])
                differences[f'{par}___{par2}']= (res, d)
    
    return differences
#--------------------------------------- Summary
def cat_df_summary(dict_differences, corrected_vals, column):
    df=  pd.DataFrame(columns=['Col1','Col2',column,'Reject_h0','Difference','Difference_#'])
    

    df[column]= corrected_vals[1]
    df['Col1']= [key.split('___')[0]for key in dict_differences.keys()]
    df['Col2']= [key.split('___')[1]for key in dict_differences.keys()]
    df['Reject_h0']=  corrected_vals[0]   
    
    df['Difference']= [v_tuple[0] for v_tuple in dict_differences.values()]
    df['Difference_#']= [v_tuple[1] for v_tuple in dict_differences.values()]

    return df

# ----------------------------------------Graphics
from matplotlib.patches import PathPatch

def adjust_box_widths(g, fac):
    """
    Adjust the withs of a seaborn-generated boxplot. 
    # From stackoverflow
    """

    # iterating through Axes instances
    for ax in g.axes:

        # iterating through axes artists:
        for c in ax.get_children():

            # searching for PathPatches
            if isinstance(c, PathPatch):
                # getting current width of box:
                p = c.get_path()
                verts = p.vertices
                verts_sub = verts[:-1]
                ymin = np.min(verts_sub[:, 1])
                ymax = np.max(verts_sub[:, 1])
                ymid = 0.5*(ymin+ymax)
                yhalf = 0.5*(ymax - ymin)

                # setting new width of box
                ymin_new = ymid-fac*yhalf
                ymax_new = ymid+fac*yhalf
                verts_sub[verts_sub[:, 1] == ymin, 1] = ymin_new
                verts_sub[verts_sub[:, 1] == ymax, 1] = ymax_new

                # setting new width of median line
                for l in ax.lines:
                    if np.all(l.get_ydata() == [ymin, ymax]):
                        l.set_ydata([ymin_new, ymax_new])
                        
def title(word):
    words = word.split("_")
    n_word= ''
    for w in words:
        if w != 'and':
            n_word = n_word +" " +w.title()
        else:
            n_word = n_word + " " + w
    return n_word

### TLO

## By country

In [ ]:
#fig= plt.figure(figsize=(9, 7))
fig = plt.figure(figsize=(10, 4))

sns.set_style("whitegrid")
ax = sns.boxplot(y="Country", x="Occurrences",
                  data=df_occurences_countries, palette=seq_col_brew[1:],fliersize=3)
 
ax.set_ylabel("Country", fontsize=18)
ax.set_xlabel("Occurences",fontsize=18)
ax.tick_params(labelsize=14)

adjust_box_widths(fig,0.7)

# plt.legend(bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0.)
plt.tight_layout()

fig.savefig("TLO_country.png")

In [ ]:
df_occurences_countries.groupby(['Country'])['Occurrences'].agg(pd.Series.mode)

In [ ]:
des_TLO_countries = df_occurences_countries.groupby('Country').describe().reset_index()
des_TLO_countries.columns = des_TLO_countries.columns.droplevel()
des_TLO_countries

In [ ]:
from scipy.stats import skew 
print(
skew(df_occurences_countries[df_occurences_countries['Country']=='CO']['Occurrences']),
skew(df_occurences_countries[df_occurences_countries['Country']=='BR']['Occurrences']),
skew(df_occurences_countries[df_occurences_countries['Country']=='DE']['Occurrences']),
skew(df_occurences_countries[df_occurences_countries['Country']=='US']['Occurrences']))


In [ ]:
p_values, u_values= manWhitney_pairs(df_occurences_countries, 'Country','Occurrences',countries, c_type='CON')
p_adjusted = multipletests(list(p_values.values()), alpha=0.05, method='holm')
key_values= dict(zip(list(p_values.keys()), p_adjusted[0]))

differences= cliffsDelta_pairs(df_occurences_countries, 'Country','Occurrences', countries,c_type='CON')
df_summary_cont= cat_df_summary(differences,p_adjusted,'p_value')


print(list(p_values.keys()))
print(p_adjusted[0])
print(p_adjusted[1])

print("\n")

print("Effect size")
print(differences)

## By top

In [ ]:
fig = plt.figure(figsize=(10, 4))
sns.set_style("whitegrid")
ax = sns.boxplot(y="Top", x="Occurrences",
                  data=df_occurences_countries, palette=seq_col_brew[1:], fliersize=3)
for box in ax.artists:
    box.set_facecolor("white")
    

ax.set_ylabel("", fontsize=1)
ax.set_xlabel("Occurences",fontsize=18)
ax.tick_params(labelsize=14)

texts = [t.get_text()  for t in ax.get_yticklabels()]

ticks = {'topFree':'Top free', 'topSelling':'Top Selling', 'editorChoice': "Editor's choice"}
ax.set_yticklabels([ ticks[t] for t in texts])


adjust_box_widths(fig,0.7)

# plt.legend(bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0.)
plt.tight_layout()

fig.savefig("TLO_top.png")

In [ ]:
des_TLO_top = df_occurences_countries.groupby('Top').describe().reset_index()
des_TLO_top.columns = des_TLO_top.columns.droplevel()
des_TLO_top

In [ ]:
print(
skew(df_occurences_countries[df_occurences_countries['Top']=='topFree']['Occurrences']),
skew(df_occurences_countries[df_occurences_countries['Top']=='topSelling']['Occurrences']),
skew(df_occurences_countries[df_occurences_countries['Top']=='editorChoice']['Occurrences']))

In [ ]:
p_values, u_values= manWhitney_pairs(df_occurences_countries,'Top','Occurrences',top)
p_adjusted = multipletests(list(p_values.values()), alpha=0.05, method='holm')
key_values= dict(zip(list(p_values.keys()), p_adjusted[0]))

differences= cliffsDelta_pairs(df_occurences_countries,'Top','Occurrences',top, key_values)
df_summary_top= cat_df_summary(differences,p_adjusted,'p_value')



print(list(p_values.keys()))
print(p_adjusted[0])
print(p_adjusted[1])

print("\n")

print("Effect size")
print(differences)

## By category 

In [ ]:
clean_cat_no_editorChoice = np.delete(clean_cat_no_editorChoice, np.where(clean_cat_no_editorChoice == "general"))

In [ ]:
plt.figure(figsize=(16, 6))
sns.set_style("whitegrid")
data = df_occurences_countries[(df_occurences_countries['Top'] != 'editorChoice') &  (df_occurences_countries['Category'] != 'general')

#                            &(df_counts_countries['Category'] != 'general')
                          ]
ax = sns.boxplot(x="Category", y="Occurrences",
                  data=data)
for box in ax.artists:
    box.set_facecolor("white")
plt.xticks(rotation=90)
plt.tight_layout()                                                                                                    

In [ ]:
des_TLO_category = data.groupby('Category').describe().reset_index()
des_TLO_category.columns = des_TLO_category.columns.droplevel()
des_TLO_category

In [ ]:
p_values, u_values= manWhitney_pairs(data,'Category','Occurrences',clean_cat_no_editorChoice)
p_adjusted = multipletests(list(p_values.values()), alpha=0.05, method='holm')
differences= cliffsDelta_pairs(data,'Category','Occurrences',clean_cat_no_editorChoice)
df_summary_categories= cat_df_summary(differences,p_adjusted,'p_value')


# print(list(p_values.keys()))
print(p_adjusted[0])
# print(p_adjusted[1])

print("\n")

print("Effect size")
# print(differences)

In [ ]:
h0_rejected= df_summary_categories[df_summary_categories['Reject_h0']]
pivot = pd.pivot_table(h0_rejected, index= ['Col1'], columns= ['Col2'], values= 'p_value')
pivot2 = pd.pivot_table(h0_rejected, index= ['Col1'], columns= ['Col2'], values= 'Difference_#')

print(set(pivot.index) - set(pivot.columns))
print(set(pivot.columns) - set(pivot.index))

In [ ]:
pivot3= h0_rejected[['Col1','Col2','Difference']].set_index(['Col1','Col2'],inplace=False)
pivot3=pivot3.unstack()
pivot3.columns = pivot3.columns.droplevel()

In [ ]:
pivot['maps_and_navigation']= np.nan
pivot['house_and_home']= np.nan
pivot['lifestyle']= np.nan
pivot['health_and_fitness']= np.nan

pivot= pivot.append(pd.Series(name='finance'))
pivot= pivot.append(pd.Series(name='food_and_drink'))

pivot.sort_index(inplace=True)
pivot= pivot.reindex(sorted(pivot.columns), axis=1)

pivot2['maps_and_navigation']= np.nan
pivot2['house_and_home']= np.nan
pivot2['lifestyle']= np.nan
pivot2['health_and_fitness']= np.nan

pivot2= pivot2.append(pd.Series(name='finance'))
pivot2= pivot2.append(pd.Series(name='food_and_drink'))

pivot2.sort_index(inplace=True)
pivot2= pivot2.reindex(sorted(pivot2.columns), axis=1)

pivot3['maps_and_navigation']= np.nan
pivot3['house_and_home']= np.nan
pivot3['lifestyle']= np.nan
pivot3['health_and_fitness']= np.nan

pivot3= pivot3.append(pd.Series(name='finance'))
pivot3= pivot3.append(pd.Series(name='food_and_drink'))

pivot3.sort_index(inplace=True)
pivot3= pivot3.reindex(sorted(pivot3.columns), axis=1)



# create square matrix 
for row in pivot.index:
    for column in pivot.index:
        if np.isnan(pivot.loc[row,column]):
            pivot.loc[row,column]= pivot.loc[column,row] 

# create square matrix 
for row in pivot2.index:
    for column in pivot2.index:
        if np.isnan(pivot2.loc[row,column]):
            pivot2.loc[row,column]= pivot2.loc[column,row] 
            
# create square matrix 
for row in pivot3.index:
    for column in pivot3.index:
        if pd.isnull(pivot3.loc[row,column]):
            pivot3.loc[row,column]= pivot3.loc[column,row] 
        if pivot3.loc[row,column] =='small':
            pivot3.loc[row,column] = 'S'
        elif pivot3.loc[row,column] =='medium':
            pivot3.loc[row,column] = 'M'
        elif pivot3.loc[row,column] =='large':
            pivot3.loc[row,column] = 'L'
        elif pivot3.loc[row,column] =='negligible':
            pivot3.loc[row,column] = 'N'

In [ ]:
sns.set(style="white")

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(pivot, dtype=np.bool))

# Set up the matplotlib figure
fig, ax = plt.subplots(figsize=(11, 10))


# cbar_ax = fig.add_axes()

cbar_ax = fig.add_axes([.85, .55, .02, .4])


# Draw the heatmap with the mask and correct aspect ratio
ax= sns.heatmap(pivot, mask=mask, cmap='Greys_r', center=0.035,
            square=True, linewidths=0.6,vmax=0.05,  vmin=0, ax=ax, cbar_ax=cbar_ax, annot = pivot3.values, fmt = '',
                cbar_kws={"shrink": .5, }, 
                annot_kws={"size": 14, })



ax.set_ylabel("", fontsize=1)
ax.set_xlabel("",fontsize=1)
ax.tick_params(labelsize=15)

texts = [t.get_text()  for t in ax.get_yticklabels()]

# ticks = {'topFree':'Top free', 'topSelling':'Top Selling', 'editorChoice': "Editor's choice"}
ax.set_yticklabels([ title(t) for t in texts])
ax.set_xticklabels([ title(t) for t in texts])





cbar_ax.tick_params(labelsize=14)

fig = ax.get_figure()
plt.tight_layout()


fig.savefig("TLO_category.png")

In [ ]:
df_summary_categories.shape

In [ ]:
df_summary_categories[df_summary_categories.Reject_h0].shape

In [ ]:
df_summary_categories[(df_summary_categories.Reject_h0)& (df_summary_categories.Difference != 'negligible')].shape

In [ ]:
x =pd.DataFrame(df_summary_categories[(df_summary_categories.Reject_h0)& (df_summary_categories.Difference != 'negligible')].Col2.value_counts()).join(pd.DataFrame(df_summary_categories[(df_summary_categories.Reject_h0)& (df_summary_categories.Difference != 'negligible')].Col1.value_counts()) )
x.Col1 + x.Col2

## EST

## By country

In [ ]:
plt.figure(figsize=(9, 7))

sns.set_style("whitegrid")
ax = sns.boxplot(x="Country", y="Count", data=df_counts_countries, palette=seq_col_brew[1:], width=0.5)

fig = ax.get_figure()
fig.savefig("EST_country.png")

In [ ]:
des_DTLO_countries = df_counts_countries.groupby('Country').describe().reset_index()
des_DTLO_countries.columns = des_DTLO_countries.columns.droplevel()
des_DTLO_countries

In [ ]:
p_values, u_values= manWhitney_pairs(df_counts_countries, 'Country','Count',countries, c_type='CON')
p_adjusted = multipletests(list(p_values.values()), alpha=0.05, method='holm')
differnces= cliffsDelta_pairs(df_counts_countries, 'Country','Count',countries, c_type='CON')

print(list(p_values.keys()))
print(p_adjusted[0])
print(p_adjusted[1])

print("\n")

print("Effect size")
print(differnces)

## By top

In [ ]:
plt.figure(figsize=(9, 7))
sns.set_style("whitegrid")
ax = sns.boxplot(x="Top", y="Count",
                  data=df_counts_countries, palette=seq_col_brew[1:],  width=0.5)
for box in ax.artists:
    box.set_facecolor("white")
fig = ax.get_figure()
fig.savefig("EST_top.png")

In [ ]:
des_DTLO_top = df_counts_countries.groupby('Top').describe().reset_index()
des_DTLO_top.columns = des_DTLO_top.columns.droplevel()
des_DTLO_top

In [ ]:
from scipy.stats import skew 

print(
skew(df_counts_countries[df_counts_countries['Top']=='topFree']['Count']),
skew(df_counts_countries[df_counts_countries['Top']=='topSelling']['Count']),
skew(df_counts_countries[df_counts_countries['Top']=='editorChoice']['Count']))

In [ ]:
p_values, u_values= manWhitney_pairs(df_counts_countries,'Top','Count',top)
p_adjusted = multipletests(list(p_values.values()), alpha=0.05, method='holm')
differences= cliffsDelta_pairs(df_counts_countries,'Top','Count',top)

print(list(p_values.keys()))
print(p_adjusted[0])
print(p_adjusted[1])

print("\n")

print("Effect size")
print(differences)

## By category 

In [ ]:
plt.figure(figsize=(16, 6))
sns.set_style("whitegrid")
data = df_counts_countries[(df_counts_countries['Top'] != 'editorChoice') & (df_counts_countries['Category'] != 'general')
#                            &(df_counts_countries['Category'] != 'general')
                          ]
ax = sns.boxplot(x="Category", y="Count",
                  data=data,  width=0.5)
for box in ax.artists:
    box.set_facecolor("white")
plt.xticks(rotation=90)
plt.tight_layout()                                                                                                    

In [ ]:
data.Category.nunique()

In [ ]:
des_DTLO_category = data.groupby('Category').describe().reset_index()
des_DTLO_category.columns = des_DTLO_category.columns.droplevel()
des_DTLO_category

In [ ]:
p_values, u_values= manWhitney_pairs(data,'Category','Count',clean_cat_no_editorChoice)
p_adjusted = multipletests(list(p_values.values()), alpha=0.05, method='holm')
differences= cliffsDelta_pairs(data,'Category','Count',clean_cat_no_editorChoice)
df_summary_categories= cat_df_summary(differences,p_adjusted,'p_value')
# print(list(p_values.keys()))
# print(p_adjusted[0])
# print(p_adjusted[1])

print("\n")

print("Effect size")
# print(differences)

In [ ]:
h0_rejected= df_summary_categories[df_summary_categories['Reject_h0']]
pivot = pd.pivot_table(h0_rejected, index= ['Col1'], columns= ['Col2'], values= 'p_value')
pivot2 = pd.pivot_table(h0_rejected, index= ['Col1'], columns= ['Col2'], values= 'Difference_#')

print(set(pivot.index) - set(pivot.columns))
print(set(pivot.columns) - set(pivot.index))

In [ ]:
pivot3= h0_rejected[['Col1','Col2','Difference']].set_index(['Col1','Col2'],inplace=False)
pivot3=pivot3.unstack()
pivot3.columns = pivot3.columns.droplevel()

In [ ]:
pivot['health_and_fitness']= np.nan
pivot= pivot.append(pd.Series(name='food_and_drink'))

pivot.sort_index(inplace=True)
pivot= pivot.reindex(sorted(pivot.columns), axis=1)

# create square matrix 
for row in pivot.index:
    for column in pivot.index:
        if np.isnan(pivot.loc[row,column]):
            pivot.loc[row,column]= pivot.loc[column,row] 
            
            
pivot2['health_and_fitness']= np.nan

           
pivot2= pivot2.append(pd.Series(name='food_and_drink'))

pivot2.sort_index(inplace=True)
pivot2= pivot2.reindex(sorted(pivot2.columns), axis=1)

pivot3['health_and_fitness']= np.nan

pivot3= pivot3.append(pd.Series(name='food_and_drink'))

pivot3.sort_index(inplace=True)
pivot3= pivot3.reindex(sorted(pivot3.columns), axis=1)



# create square matrix 
for row in pivot.index:
    for column in pivot.index:
        if np.isnan(pivot.loc[row,column]):
            pivot.loc[row,column]= pivot.loc[column,row] 

# create square matrix 
for row in pivot2.index:
    for column in pivot2.index:
        if np.isnan(pivot2.loc[row,column]):
            pivot2.loc[row,column]= pivot2.loc[column,row] 
            
# create square matrix 
for row in pivot3.index:
    for column in pivot3.index:
        if pd.isnull(pivot3.loc[row,column]):
            pivot3.loc[row,column]= pivot3.loc[column,row] 
        if pivot3.loc[row,column] =='small':
            pivot3.loc[row,column] = 'S'
        elif pivot3.loc[row,column] =='medium':
            pivot3.loc[row,column] = 'M'
        elif pivot3.loc[row,column] =='large':
            pivot3.loc[row,column] = 'L'
        elif pivot3.loc[row,column] =='negligible':
            pivot3.loc[row,column] = 'N'

In [ ]:
sns.set(style="white")

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(pivot, dtype=np.bool))

# Set up the matplotlib figure
fig, ax = plt.subplots(figsize=(11, 10))


# cbar_ax = fig.add_axes()

cbar_ax = fig.add_axes([.85, .55, .02, .4])


# Draw the heatmap with the mask and correct aspect ratio
ax= sns.heatmap(pivot, mask=mask, cmap='Greys_r', center=0.035,
            square=True, linewidths=0.6,vmax=0.05,  vmin=0, ax=ax, cbar_ax=cbar_ax, annot = pivot3.values, fmt = '',
                cbar_kws={"shrink": .5, }, 
                annot_kws={"size": 14, })



ax.set_ylabel("", fontsize=1)
ax.set_xlabel("",fontsize=1)
ax.tick_params(labelsize=15)

texts = [t.get_text()  for t in ax.get_yticklabels()]

# ticks = {'topFree':'Top free', 'topSelling':'Top Selling', 'editorChoice': "Editor's choice"}
ax.set_yticklabels([ title(t) for t in texts])
ax.set_xticklabels([ title(t) for t in texts])





cbar_ax.tick_params(labelsize=14)

fig = ax.get_figure()
plt.tight_layout()

fig.savefig("EST_category.png")

In [ ]:
df_summary_categories[(df_summary_categories.Reject_h0)].shape

In [ ]:
df_summary_categories[(df_summary_categories.Reject_h0)& (df_summary_categories.Difference != 'negligible')].shape

In [ ]:
x = pd.DataFrame(df_summary_categories[(df_summary_categories.Reject_h0)& (df_summary_categories.Difference != 'negligible')].Col1.value_counts()).join( pd.DataFrame(df_summary_categories[(df_summary_categories.Reject_h0)& (df_summary_categories.Difference != 'negligible')].Col2.value_counts()))
x.fillna(0, inplace=True)
x['Col1'] +  x['Col2']

### TBSE

## By country

In [ ]:
df_time_between_countries.columns

In [ ]:
plt.figure(figsize=(16, 10))
sns.set_style("whitegrid")
ax = sns.boxplot(x="Country", y="Between_apps",
                  data=df_time_between_countries, palette=seq_col_brew[1:])

In [ ]:
des_DBTLO_countries = df_time_between_countries.groupby('Country').describe().reset_index()
des_DBTLO_countries.columns = des_DBTLO_countries.columns.droplevel()
des_DBTLO_countries

In [ ]:
p_values, u_values= manWhitney_pairs(df_time_between_countries, 'Country','Between_apps',countries, c_type='CON')
p_adjusted = multipletests(list(p_values.values()), alpha=0.05, method='holm')
differnces= cliffsDelta_pairs(df_time_between_countries, 'Country','Between_apps',countries, c_type='CON')

print(list(p_values.keys()))
print(p_adjusted[0])
print(p_adjusted[1])

print("\n")

print("Effect size")
print(differnces)

## By top

In [ ]:
plt.figure(figsize=(9, 7))

sns.set_style("whitegrid")
ax = sns.boxplot(x="Top", y="Between_apps",
                  data=df_time_between_countries, palette=seq_col_brew[1:])
for box in ax.artists:
    box.set_facecolor("white")
    
fig = ax.get_figure()
fig.savefig("TBSE_top.png")

In [ ]:
des_DBTLO_top = df_time_between_countries.groupby('Top').describe().reset_index()
des_DBTLO_top.columns = des_DBTLO_top.columns.droplevel()
des_DBTLO_top

In [ ]:
from scipy.stats import skew 

print(
skew(df_time_between_countries[df_time_between_countries['Top']=='topFree']['Between_apps']),
skew(df_time_between_countries[df_time_between_countries['Top']=='topSelling']['Between_apps']),
skew(df_time_between_countries[df_time_between_countries['Top']=='editorChoice']['Between_apps']))

In [ ]:
p_values, u_values= manWhitney_pairs(df_time_between_countries,'Top','Between_apps',top)
p_adjusted = multipletests(list(p_values.values()), alpha=0.05, method='holm')
differences= cliffsDelta_pairs(df_time_between_countries,'Top','Between_apps',top)


print(list(p_values.keys()))
print(p_adjusted[0])
print(p_adjusted[1])

print("\n")

print("Effect size")
print(differences)

## By category 

In [ ]:
plt.figure(figsize=(16, 6))
sns.set_style("whitegrid")
data = df_time_between_countries[(df_time_between_countries['Top'] != 'editorChoice') & (df_time_between_countries['Category'] != 'general')

#                            &(df_counts_countries['Category'] != 'general')
                          ]
ax = sns.boxplot(x="Category", y="Between_apps",
                  data=data)
for box in ax.artists:
    box.set_facecolor("white")
plt.xticks(rotation=90)
plt.tight_layout()                                                                                                    

In [ ]:
des_DBTLO_category = data.groupby('Category').describe().reset_index()
des_DBTLO_category.columns = des_DBTLO_category.columns.droplevel()
des_DBTLO_category

In [ ]:
p_values, u_values= manWhitney_pairs(data,'Category','Between_apps',clean_cat_no_editorChoice)
p_adjusted = multipletests(list(p_values.values()), alpha=0.05, method='holm')
differences= cliffsDelta_pairs(data,'Category','Between_apps',clean_cat_no_editorChoice)
df_summary_categories= cat_df_summary(differences,p_adjusted,'p_value')
# print(list(p_values.keys()))
# print(p_adjusted[0])
# print(p_adjusted[1])

# print("\n")

# print("Effect size")
# print(differences)

In [ ]:
h0_rejected= df_summary_categories[df_summary_categories['Reject_h0']]
pivot = pd.pivot_table(h0_rejected, index= ['Col1'], columns= ['Col2'], values= 'p_value')
pivot2 = pd.pivot_table(h0_rejected, index= ['Col1'], columns= ['Col2'], values= 'Difference_#')

print(set(pivot.index) - set(pivot.columns))
print(set(pivot.columns) - set(pivot.index))

In [ ]:
pivot3= h0_rejected[['Col1','Col2','Difference']].set_index(['Col1','Col2'],inplace=False)
pivot3=pivot3.unstack()
pivot3.columns = pivot3.columns.droplevel()

In [ ]:
pivot['libraries_and_demo']= np.nan
pivot['health_and_fitness']= np.nan

pivot= pivot.append(pd.Series(name='finance'))
pivot= pivot.append(pd.Series(name='food_and_drink'))
pivot= pivot.append(pd.Series(name='comics'))
pivot= pivot.append(pd.Series(name='entertainment'))
pivot= pivot.append(pd.Series(name='education'))

pivot.sort_index(inplace=True)
pivot= pivot.reindex(sorted(pivot.columns), axis=1)

# create square matrix 
for row in pivot.index:
    for column in pivot.index:
        if np.isnan(pivot.loc[row,column]):
            pivot.loc[row,column]= pivot.loc[column,row] 
            
            
pivot2['libraries_and_demo']= np.nan
pivot2['health_and_fitness']= np.nan


pivot2= pivot2.append(pd.Series(name='finance'))
pivot2= pivot2.append(pd.Series(name='food_and_drink'))
pivot2= pivot2.append(pd.Series(name='comics'))
pivot2= pivot2.append(pd.Series(name='entertainment'))
pivot2= pivot2.append(pd.Series(name='education'))


pivot2.sort_index(inplace=True)
pivot2= pivot2.reindex(sorted(pivot2.columns), axis=1)

pivot3['libraries_and_demo']= np.nan
pivot3['health_and_fitness']= np.nan


pivot3= pivot3.append(pd.Series(name='finance'))
pivot3= pivot3.append(pd.Series(name='food_and_drink'))
pivot3= pivot3.append(pd.Series(name='comics'))
pivot3= pivot3.append(pd.Series(name='entertainment'))
pivot3= pivot3.append(pd.Series(name='education'))


pivot3.sort_index(inplace=True)
pivot3= pivot3.reindex(sorted(pivot3.columns), axis=1)



# create square matrix 
for row in pivot.index:
    for column in pivot.index:
        if np.isnan(pivot.loc[row,column]):
            pivot.loc[row,column]= pivot.loc[column,row] 

# create square matrix 
for row in pivot2.index:
    for column in pivot2.index:
        if np.isnan(pivot2.loc[row,column]):
            pivot2.loc[row,column]= pivot2.loc[column,row] 
            
# create square matrix 
for row in pivot3.index:
    for column in pivot3.index:
        if pd.isnull(pivot3.loc[row,column]):
            pivot3.loc[row,column]= pivot3.loc[column,row] 
        if pivot3.loc[row,column] =='small':
            pivot3.loc[row,column] = 'S'
        elif pivot3.loc[row,column] =='medium':
            pivot3.loc[row,column] = 'M'
        elif pivot3.loc[row,column] =='large':
            pivot3.loc[row,column] = 'L'
        elif pivot3.loc[row,column] =='negligible':
            pivot3.loc[row,column] = 'N'

In [ ]:
sns.set(style="white")

# Generate a mask for the upper triangle
mask = np.triu(np.ones_like(pivot, dtype=np.bool))

# Set up the matplotlib figure
fig, ax = plt.subplots(figsize=(11, 10))


# cbar_ax = fig.add_axes()

cbar_ax = fig.add_axes([.85, .55, .02, .4])


# Draw the heatmap with the mask and correct aspect ratio
ax= sns.heatmap(pivot, mask=mask, cmap='Greys_r', center=0.035,
            square=True, linewidths=0.6,vmax=0.05,  vmin=0, ax=ax, cbar_ax=cbar_ax, annot = pivot3.values, fmt = '',
                cbar_kws={"shrink": .5, }, 
                annot_kws={"size": 14, })

ax.set_ylabel("", fontsize=1)
ax.set_xlabel("",fontsize=1)
ax.tick_params(labelsize=15)

texts = [t.get_text()  for t in ax.get_yticklabels()]

# ticks = {'topFree':'Top free', 'topSelling':'Top Selling', 'editorChoice': "Editor's choice"}
ax.set_yticklabels([ title(t) for t in texts])
ax.set_xticklabels([ title(t) for t in texts])

cbar_ax.tick_params(labelsize=14)

fig = ax.get_figure()
plt.tight_layout()

fig.savefig("TBSE_category.png")


In [ ]:
df_summary_categories[(df_summary_categories.Reject_h0)].shape

In [ ]:
df_summary_categories[(df_summary_categories.Reject_h0)& (df_summary_categories.Difference != 'negligible')].shape

In [ ]:
x = pd.DataFrame(df_summary_categories[(df_summary_categories.Reject_h0)& (df_summary_categories.Difference != 'negligible')].Col1.value_counts()).join( pd.DataFrame(df_summary_categories[(df_summary_categories.Reject_h0)& (df_summary_categories.Difference != 'negligible')].Col2.value_counts()))
x.fillna(0, inplace=True)
x['Col1'] +  x['Col2']

### EST and TBSE for Country and Top aggregation

In [ ]:
df_occurences_countries2 = df_occurences_countries.copy()
df_occurences_countries2.columns = ['Top', 'Category', '# Events', 'Country', 'Type']
df_counts_countries2 = df_counts_countries.copy()
df_counts_countries2.columns = ['Top', 'Category', 'Weeks', 'Country', 'Event']
df_time_between_countries2 = df_time_between_countries.copy()
df_time_between_countries2.columns = ['Top', 'Category', 'Weeks', 'Country', 'Event']

In [ ]:
duration = pd.concat([df_time_between_countries2, df_counts_countries2], ignore_index=True)

In [ ]:
fig= plt.figure(figsize=(9, 6))

sns.set_style("whitegrid")
ax = sns.boxplot(y="Event", x="Weeks", hue='Country',
                  data=duration, palette=seq_col_brew[1:], order= ['EST','TBSE'])

ax.set_xlabel("Weeks", fontsize=15)
ax.set_ylabel("Event",fontsize=15)
ax.tick_params(labelsize=15)
adjust_box_widths(fig, 0.7)
# fig.savefig("DTLO_country.png")
plt.legend(bbox_to_anchor=(1.01, 1), loc=2, borderaxespad=0., fontsize='15')

fig = ax.get_figure()
fig.savefig("country_event.png")

In [ ]:
fig= plt.figure(figsize=(10, 6))

sns.set_style("whitegrid")
ax = sns.boxplot(y="Event", x="Weeks",  hue='Top',
                  data=duration, palette='Greys', fliersize=3, order= ['EST','TBSE'])

adjust_box_widths(fig, 0.7)

ax.set_ylabel("", fontsize=1)
ax.set_xlabel("Weeks",fontsize=18)
ax.tick_params(labelsize=14)

h, l = ax.get_legend_handles_labels()
plt.legend(handles=h, labels=['Top selling', 'Top free', "Editor's choice"], fontsize='14')


plt.tight_layout()
fig = ax.get_figure()
fig.savefig("top_events.png")
